# Flows with friction (Fanno flows)

In [10]:
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np
from scipy.optimize import root_scalar

# Pint gives us some helpful unit conversion
from pint import UnitRegistry
ureg = UnitRegistry()
Q_ = ureg.Quantity # We will use this to construct quantities (value + unit)

In [11]:
# these lines are only for helping improve the display
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'png')
plt.rcParams['figure.dpi']= 200
plt.rcParams['savefig.dpi'] = 200

After area change, friction is another important factor that controls compressible flow.
(Heat transfer is the third main factor.) 
Of course, in reality more than one of these factors can play a role at the same time—we'll examine those cases later.

**Fanno flow** is the specific case of adiabatic flow in a constant-area duct, with effects due to friction.

## Fanno flow theory

For a general fluid, we can apply conservation of mass: $ \dot{m} = \rho A V $, but since area is constant, we can introduce the **mass velocity**:

$$
G = \rho V = \text{constant} \;.
$$

Next, applying conservation of energy:

$$
\begin{gather*}
h_{t1} + q = h_{t2} + w_s = h_t \\
\rightarrow h_t = h + \frac{V^2}{2} \\
V^2 = \left( \frac{G}{\rho} \right)^2 \rightarrow h_t = h + \frac{G^2}{2 \rho^2} = \text{constant} \;.
\end{gather*}
$$

So, we have a relationship between $h$ and $\rho$ (or $v$) in a Fanno flow.

## Fanno flow analysis


$$
\frac{T_2}{T_1} = \frac{2 + (\gamma-1) M_1^2}{2 + (\gamma-1) M_2^2}
$$ (eq_fanno_temperature)

$$
\frac{p_2}{p_1} = \frac{M_1}{M_2} \left( \frac{2 + (\gamma-1) M_1^2}{2 + (\gamma-1) M_2^2} \right)^{1/2}
$$ (eq_fanno_pressure)

$$
\frac{\rho_2}{\rho_1} = \frac{M_1}{M_2} \left[ \frac{2 + (\gamma-1) M_2^2}{2 + (\gamma-1) M_1^2} \right]^{1/2}
$$ (eq_fanno_density)

$$
\frac{p_{t2}}{p_{t1}} = \frac{M_1}{M_2} \left( \frac{2 + (\gamma-1) M_2^2}{2 + (\gamma-1) M_1^2} \right)^{\frac{\gamma+1}{2(\gamma-1)}}
$$ (eq_fanno_stag_pressure)

$$
\frac{f \Delta x}{D_e} = \frac{\gamma+1}{2\gamma} \log \left[ \frac{2 + (\gamma-1) M_2^2}{2 + (\gamma-1) M_1^2} \right] - \frac{1}{\gamma} \left( \frac{1}{M_2^2} - \frac{1}{M_1^2} \right) - \frac{\gamma+1}{2 \gamma} \log \frac{M_2^2}{M_1^2}
$$ (eq_fanno_length)

In [29]:
def get_fanno_distance(mach_1, mach_2, gamma=1.4):
    '''Get Fanno friction distance (f Δx/D) given two Mach numbers'''
    return (
        ((gamma + 1)/(2 * gamma)) *
        np.log((2 + (gamma - 1)*mach_2**2) / (2 + (gamma - 1)*mach_1**2)) -
        (1/gamma)*(1/mach_2**2 - 1/mach_1**2) -
        (gamma + 1) * np.log(mach_2**2 / mach_1**2) / (2*gamma)
        )

def solve_fanno_mach(mach_2, mach_1, friction_factor, delta_x, diameter, gamma=1.4):
    '''Use to solve for downstream Mach in Fanno flow'''
    return (
        friction_factor*delta_x/diameter - get_fanno_distance(mach_1, mach_2, gamma)
        )

def get_fanno_temperature_ratio(mach_1, mach_2, gamma=1.4):
    '''Return T2/T1 for Fanno flow'''
    return (
        (2 + (gamma-1)*mach_1**2) / (2 + (gamma-1)*mach_2**2)
        )

def get_fanno_pressure_ratio(mach_1, mach_2, gamma=1.4):
    '''Return p2/p1 for Fanno flow'''
    return (
        (mach_1 / mach_2) * 
        np.sqrt((2 + (gamma-1)*mach_1**2)/(2 + (gamma-1)*mach_2**2))
        )

def get_fanno_stag_pressure_ratio(mach_1, mach_2, gamma=1.4):
    '''Return pt2/pt1 for Fanno flow'''
    return (
        (mach_1 / mach_2) * (
            (2 + (gamma-1)*mach_2**2) / 
            (2 + (gamma-1)*mach_1**2)
            )**((gamma + 1) / (2*(gamma - 1)))
        )

## Fanno sonic reference state

The * reference state for a Fanno flow is the state that would exist if the flow continued until the Mach number is 1.0, through additional length of duct (with friction).
Since this is the limiting point on a Fanno line, all states in a given Fanno flow share the same * reference state.

$$
\frac{f L_{\max}}{D_e} = \left( \frac{\gamma+1}{2\gamma} \right) \log \left( \frac{\gamma+1}{2 + (\gamma-1) M^2} \right) - \frac{1}{\gamma} \left( 1 - \frac{1}{M^2} \right) - \left( \frac{\gamma+1}{2\gamma} \right) \log \left(\frac{1}{M^2}\right)
$$

where $L_{\max}$ = $x^* - x$ is the maximum length of duct allowable on the Fanno line from the given state before reaching the limiting point. 

$$
\frac{T}{T^*} = \frac{\gamma+1}{2 + (\gamma-1) M^2}
$$

$$
\frac{p}{p^*} = \frac{1}{M} \left[ \frac{\gamma+1}{2 + (\gamma-1)M^2}\right]^{1/2}
$$

$$
\frac{\rho}{\rho^*} = \frac{1}{M} \left[ \frac{2 + (\gamma-1) M^2}{\gamma+1}\right]^{1/2}
$$

$$
\frac{p_t}{p_t^*} = \frac{1}{M} \left[ \frac{2 + (\gamma-1) M^2}{\gamma+1}\right]^{\frac{\gamma+1}{2(\gamma-1)}}
$$

In [30]:
def get_fanno_max_length(mach, gamma=1.4):
    '''Get maximum Fanno friction length (f L_max / D) given Mach number'''
    return (
        ((gamma + 1)/(2 * gamma)) *
        np.log((gamma + 1) / (2 + (gamma - 1)*mach**2)) -
        (1/gamma)*(1 - 1/mach**2) -
        (gamma + 1) * np.log(1 / mach**2) / (2*gamma)
        )

def solve_fanno_reference_mach(mach, friction_factor, L_max, diameter, gamma=1.4):
    '''Use to solve for downstream Mach in Fanno flow'''
    return (
        friction_factor*L_max/diameter - get_fanno_max_length(mach, gamma)
        )

def get_fanno_reference_pressure(mach, gamma=1.4):
    '''Return p/p* for Fanno flow'''
    return (
        (1 / mach) * 
        np.sqrt((gamma + 1) / (2 + (gamma-1)*mach**2))
        )

## Example: constant-area duct following nozzle

Consider a system with an isentropic converging-diverging nozzle that flows into a 2.4 m section of constant-diameter duct (with diameter 0.14 m), exhausting into 1 bar pressure. The area ratio of the nozzle is 5.42, and the friction factor in the constant-diameter section if 0.02.

Find the minimum stagnation pressure in the supply for the nozzle if the flow is all supersonic in the system, starting at the throat.

First, we need to determine the supersonic flow operation of the nozzle (the third critical). For an area ratio $A_3 / A_2$, we can solve for the outlet Mach number $M_3$ and pressure ratio $p_3 / p_{t3}$:

In [31]:
# isentropic relationships
def get_reference_area(mach, gamma):
    '''Calculate reference area ratio'''
    return ((1.0/mach) * ((1 + 0.5*(gamma-1)*mach**2) / 
            ((gamma + 1)/2))**((gamma+1) / (2*(gamma-1)))
            )

def solve_mach_area(mach, area_ratio, gamma):
    '''Used to find Mach number for given reference area ratio and gamma'''
    return (area_ratio - get_reference_area(mach, gamma))

def get_stagnation_pressure_ratio(mach, gamma):
    '''Calculate p/pt given Mach number'''
    return (1 + 0.5*(gamma - 1)*mach**2)**(-gamma/(gamma - 1))

In [32]:
gamma = 1.4
area_ratio = 5.42
pt1_pt3 = 1.0 # isentropic nozzle

diameter = Q_(0.14, 'm')
f = 0.02
p_rec = Q_(1, 'bar')
length = Q_(2.4, 'm')

root = root_scalar(solve_mach_area, x0=2.0, x1=3.0, args=(area_ratio, gamma))
M3 = root.root
print(f'Nozzle outlet M3 = {M3: .3f}')

p3_pt3 = get_stagnation_pressure_ratio(M3, gamma)
print(f'p3/pt3 = {p3_pt3: .4f}')

Nozzle outlet M3 =  3.260
p3/pt3 =  0.0185


Next, for the length of duct with friction, the inlet and outlet share the same Fanno sonic reference state, so

$$
L_{3, \max} = \Delta x + L_{4, \max}
$$

and we can then find $M_4$ using $\frac{f L_{4, \max}}{D}$:

In [33]:
fL3_max_D = get_fanno_max_length(M3, gamma)
fL4_max_D = fL3_max_D - (f * length / diameter)
L4_max = fL4_max_D * diameter / f

root = root_scalar(
    solve_fanno_reference_mach, x0=2, x1=3, 
    args=(f, L4_max, diameter, gamma)
    )
M4 = root.root
print(f'M4 = {M4: .3f}')

p3_p3star = get_fanno_reference_pressure(M3, gamma)
p4_p4star = get_fanno_reference_pressure(M4, gamma)
p3star_p4star = 1.0

# p4 = p_rec
pt1 = pt1_pt3 * (1/p3_pt3) * p3_p3star * p3star_p4star * (1/p4_p4star) * p_rec
print(f'Supply stagnation pressure: {pt1.to("bar"): .2f}')

M4 = 1.722 dimensionless
Supply stagnation pressure: 20.35 bar
